In [94]:
import numpy as np

In [95]:
DAYS = {"M": "000", "T": "001", "W": "010", "TH": "011", "F":"100"}

In [96]:
DAYS_INV = dict((v, k) for k, v in DAYS.items())

In [97]:
TIMES = {8: "0000", 9: "0001", 10: "0010", 11: "0011", 12: "0100", 13: "0101", 14: "0110", 15: "0111", 16: "1000"}

In [98]:
TIMES_INV = dict((v, k) for k, v in TIMES.items())

In [99]:
LENGTHS = {1: "00", 1.5: "01", 2: "10"}

In [203]:
LENGTHS_INV = dict((v, k) for k, v in LENGTHS.items())

In [101]:
ROOMS = {"R1": "0000", "R2": "0001", "R3": "0010", "R4": "0011", "R5": "0100", "R6": "0101", "R7": "0110", "R8": "0111", "R9": "1000"}

In [102]:
TEACHERS = {"T1": "0000", "T2": "0001", "T3": "0010", "T4": "0011", "T5": "0100", "T6": "0101", "T7": "0110", "T8": "0111", "T9": "1000"}

In [103]:
CLASS = {"C1": "0000", "C2": "0001", "C3": "0010", "C4": "0011", "C5": "0100", "C6": "0101", "C7": "0110", "C8": "0111", "C9": "1000"}

In [104]:
DECODE = {"DAYS": DAYS, "ROOMS": ROOMS, "TIMES": TIMES, "TEACHERS": TEACHERS, "CLASS": CLASS}

In [115]:
GENE_LENGTHS = {"DAYS": len(DAYS["M"]), "TIMES": len(TIMES[8]), "ROOMS": len(ROOMS["R1"]), "TEACHERS": len(TEACHERS["T1"]),
                "CLASS": len(CLASS["C1"]), "LENGTHS": len(LENGTHS[1])}

In [106]:
ITEM_LEN = np.sum([x for x in GENE_LENGTHS.values()])
print(ITEM_LEN)

19


In [204]:
class BinFuncs:
    @staticmethod
    def get_slot(s1):
        time_len = GENE_LENGTHS["TIMES"]
        day_len = GENE_LENGTHS["DAYS"]
        length_len = GENE_LENGTHS["LENGTHS"]
        
        day_1 = DAYS_INV[s1[:day_len]]
        time_1 = TIMES_INV[s1[day_len:day_len+time_len]]
        length_1 = LENGTHS_INV[s1[day_len+time_len:day_len+time_len+length_len]]
                    
#         day_2 = DAYS_INV[s2[:day_len]]
#         time_2 = TIMES_INV[s2[day_len:day_len+time_len]]
#         length_2 = 2
#         end_2 = time_2 + length_2
        
        return (day_1, time_1, length_1)
    
    def is_overlapping(tuple1, tuple2):
        day_1, time_1, length_1 = tuple1
        day_2, time_2, length_2 = tuple2
        return Event(day_1, time_1, length_1).is_overlapping(Event(day_2, time_2, length_2))
        

In [205]:
class Event:
    def __init__(self, day, start, length):
        self.start = int(start)
        self.len = int(length)
        self.day = day
        
        self.teacher = "T1"
        self.class_name = "C1"
        self.room = "R1"
        
    def is_overlapping(self, event):
        end = self.start + self.len
        other_end = event.start + event.len
        
        end_during = end > event.start and end <= other_end
        start_during = self.start >= event.start and self.start < other_end
        same_day = (self.day == event.day)
        
        return (end_during or start_during) and same_day

    
    def __str__(self):
        return f"Starts at {self.start} for {self.len} hours on {self.day}"
    
    def time_gene(self):
        return DAYS[self.day] + TIMES[self.start] + LENGTHS[self.len]
    
    def get_genome(self):
        genome = ""
        genome += self.time_gene()
        genome += CLASS[self.class_name]
        genome += TEACHERS[self.teacher]
        return genome

In [206]:
# print(Event(15, 2.6, "M").is_overlapping(Event(12, 3.1, "W")))

In [207]:
class Schedule:
    def __init__(self, start, end, numSlots):
        self.schedule = []
        
        class_len = (end-start)/numSlots
        
        for day in DAYS.keys():

            start_time = start
            for slot in range(numSlots):
#                 print(f"Start {start_time} End: {start_time+class_len}")
                ts = Event(day, start_time, class_len)
                start_time = start_time + class_len
                self.schedule.append(ts)
            
    def __str__(self):
        rep = ""
        for event in self.schedule:
            rep = rep + str(event) + " "
            rep = rep + "\n"
        return rep
    
    def print_genome(self):
        genome = ""
        for event in self.schedule:
            print(event.get_genome())
            
    def get_as_bin(self):
        genome = ""
        for event in self.schedule:
            genome += event.get_genome()
        return genome

In [225]:
# Rough implementation, doesn't really work yet
from collections import defaultdict
def inverse_fitness(s_bin, ITEM_LEN):
    
    total_conflicts = 0
    days = defaultdict(list)
    index = 0
    day_len = GENE_LENGTHS["DAYS"]
    time_len = GENE_LENGTHS["TIMES"]
    length_len = GENE_LENGTHS["LENGTHS"]
    
    while(index < len(s_bin)):
        day = s_bin[index:index+day_len]
        time = s_bin[index+day_len: index+day_len+time_len]
        length = s_bin[index+day_len+time_len:index+day_len+time_len+length_len]
        index = index + ITEM_LEN
        days[day].append(time+length)
    print(days)
    
    for key in days.keys():
        events = days[key]
        
        for i in range(1, len(events)):
            event_one = BinFuncs.get_slot(key+events[i])
            print(f"Outer {event_one} index {i}")
            
            for j in range(i-1, -1, -1):
                event_two = BinFuncs.get_slot(key+events[j])
                print(f"Inner {event_two} index {j}")
                conflict = BinFuncs.is_overlapping(event_one, event_two)
                if conflict:
                    print("conflict")
                    total_conflicts += 1
#     for key in days.keys():
#         days[key] = sorted(days[key])
        
#         ongoing = 0
#         for event in days[key]:
            
    
#     for event s.schedule:
#         print(eventOne)
#         for eventTwo in s.schedule:
#             if eventOne != eventTwo:
#                 if eventOne.is_overlapping(eventTwo):
#                     total_conflicts+= 1
    return 2 if total_conflicts == 0 else 1./total_conflicts

In [232]:
s = Schedule(8, 16, 4)
# s.schedule = []
s.schedule.append(Event("M", 12, 2))
s.schedule.append(Event("F", 12, 2))
# s.schedule.append(Event("M", 8, 1))
# s.schedule.append(Event("M",8, 2))
# s.schedule.append(Event("M",8, 2))

# s.schedule.append(Event("T",8, 2))
# s.schedule.append(Event("T",8, 2))
print(s)

print(inverse_fitness(s.get_as_bin(), len(s.schedule[0].get_genome())))

Starts at 8 for 2 hours on M 
Starts at 10 for 2 hours on M 
Starts at 12 for 2 hours on M 
Starts at 14 for 2 hours on M 
Starts at 8 for 2 hours on T 
Starts at 10 for 2 hours on T 
Starts at 12 for 2 hours on T 
Starts at 14 for 2 hours on T 
Starts at 8 for 2 hours on W 
Starts at 10 for 2 hours on W 
Starts at 12 for 2 hours on W 
Starts at 14 for 2 hours on W 
Starts at 8 for 2 hours on TH 
Starts at 10 for 2 hours on TH 
Starts at 12 for 2 hours on TH 
Starts at 14 for 2 hours on TH 
Starts at 8 for 2 hours on F 
Starts at 10 for 2 hours on F 
Starts at 12 for 2 hours on F 
Starts at 14 for 2 hours on F 
Starts at 12 for 2 hours on M 
Starts at 12 for 2 hours on F 

defaultdict(<class 'list'>, {'000': ['000010', '001010', '010010', '011010', '010010'], '001': ['000010', '001010', '010010', '011010'], '010': ['000010', '001010', '010010', '011010'], '011': ['000010', '001010', '010010', '011010'], '100': ['000010', '001010', '010010', '011010', '010010']})
Outer ('M', 10, 2) inde

In [233]:
print(len("00000001000000000"))

17
